In [1]:
import matplotlib.pyplot as plt

import numpy as np

import pandas as pd

from zse.collections import *
from zse.utilities import *
from zse.rings import *

from matplotlib import cm

from collections import Counter
from collections import defaultdict as dd

from itertools import permutations as perm

from ase.visualize import view

In [2]:
def get_fw_data(file,code):
    file = open(file,'r')
    data = file.readlines()
    file.close()
    
    for i,line in enumerate(data):
        fields = line.split()
        if fields[0] == code:
            start = i
            break
    
    # go through fw chunk and get data
    
    t_sites = {}
    o_sites = {}
    for i,line in enumerate(data[start+1:]):
        line = line.rstrip(' \n')
        
        # check if the rings exist
        
#         if 'FAILED' in line or 'LARGE' in line:
#             return(False,False)
#         fields = line.split()
        
        # get the t site rings
        
        if 'T' in line and ':' in line:
            fields = line.split(':')
            t_sites[fields[0]]=fields[1]
#         for x in t_sites:
#             r = t_sites[x].split('_')
#             if len(r)>20:
#                 return(False,False)
                
        
        # get the o site rings
        if 'O' in line and ':' in line:
            fields = line.split(':')
            o_sites[fields[0]]=fields[1]
#         for x in o_sites:
#             r = o_sites[x]
#             if len(r)>20:
#                 return(False,False)
        
        
        
        fields = line.split()
        if fields[0] == 'Framework':
            end = i+start-1
            break

    return(t_sites,o_sites)    

In [8]:
t_sites,o_sites = get_fw_data('sastre_rings.txt','IWW')
print(t_sites)
print(o_sites)

{'T1': '4_4_4_5_6_10_12_12_12_12', 'T2': '4_4_4_6_6_6_10_10_10_10_12_12_12_12_12', 'T3': '4_4_4_6_6_6_8_10_10_12', 'T4': '4_4_4_5_6_8_10', 'T5': '5_5_5_6_6_6_6_8_8_10_10_12', 'T6': '5_5_5_5_5_6_10_12_12_12_12_12_12_12', 'T7': '5_5_5_5_6_8_8_8_8_10', 'T8': '4_5_5_6_6_6_6_10_10_10_10_12_12_12_12_12_12_12_12', 'T9': '4_5_5_5_8_8_8_8_10', 'T10': '4_5_5_5_5_5_8_8', 'T11': '4_5_5_5_6_6_10_12_12_12_12_12_12_12_12', 'T12': '4_4_5_5_6_6_10_10_12_12_12_12_12_12_12_12', 'T13': '4_5_5_5_5_6_6_12_12_12_12_12_12', 'T14': '4_5_5_6_6_6_6_6_6_12_12_12_12_12_12', 'T15': '4_5_5_6_6_6_6_6_6_8_8', 'T16': '4_5_5_5_5_6_6_8_8'}
{'O1': '4_4_6_12_12', 'O2': '4_4_10_12_12', 'O3': '4_4_5', 'O4': '5_6_10_12_12_12_12', 'O5': '4_4_10_10_10_10_12_12_12', 'O6': '4_4_6_6', 'O7': '6_6_6_10_10_10_10_12_12_12_12_12', 'O8': '4_4_10_10_12', 'O9': '4_4_6_8', 'O10': '6_6_6_8_10_10_12', 'O11': '4_4_10', 'O12': '5_6_8_10', 'O13': '4_6_6_12_12_12_12_12_12', 'O14': '4_5_5', 'O15': '5_5_5_6_12_12_12', 'O16': '4_6_6_6_6', 'O17': '5

In [2]:
s = 'AFG'
z = framework(s)
#ts = [atom.index for atom in z if atom.symbol=='Si']
ts, tm, ti = get_tsites(s)

def ov(f,ti,max_ring = 10):

    s = f
    z = framework(s)
    
    ''' 
    First get the rings
    '''
    c,r,ra,a = get_rings(z,ti,validation='vertex',max_ring=max_ring)
    oxygens = []
    o_pair_paths = dd(list)
    o_pair_sizes = dd(int)
    o_pair_counts = dd(lambda: 0)
    o_pair_weights = dd(lambda: 0)


    '''
    Find the unique oxygens attached to the T-site.
    Find the opposite angle rings, and sort them by size.
    '''
    for path in r:
        if path[1] not in oxygens:
            oxygens.append(path[1])
        if path[-1] not in oxygens:
            oxygens.append(path[-1])
        oxys = sorted([str(path[1]),str(path[-1])])
        o_pair_paths['-'.join(oxys)].append(path)
        o_pair_sizes['-'.join(oxys)] = len(path)
        o_pair_counts['-'.join(oxys)] += 1
        o_pair_weights['-'.join(oxys)] += len(path)


    perms = [x for x in perm(oxygens)]
    weights = []
    order = [[0,2],[0,1],[1,2],[2,3],[3,0],[1,3]]
    for p in perms:
        w = []
        for o in order:
            k = '-'.join(sorted([str(p[o[0]]),str(p[o[1]])]))
            w.append(o_pair_weights[k])
        weights.append(w)

    zipped_lists = zip(weights,perms)
    sp = sorted(zipped_lists,reverse=True)
    tuples = zip(*sp)
    weights, perms = [list(tuple) for tuple in tuples]    


    new_r = []
    counts = []
    sizes = []
    oxygens = perms[0]
    weights = weights[0]

    for o in order:
        k = '-'.join(sorted([str(oxygens[o[0]]),str(oxygens[o[1]])]))
        sizes.append(o_pair_sizes[k])
        counts.append(o_pair_counts[k])
        for x in o_pair_paths[k]:
            new_r.append(x)        

    ordered_v = []
    for c,s in zip(counts,sizes):
        if c == 1:
            ordered_v.append('{0}'.format(int(s/2)))
        elif c == 0:
            ordered_v.append('*')
        else:
            ordered_v.append('{0}_{1}'.format(int(s/2),c))
    ordered_v = '•'.join(ordered_v)
    c = [int(len(x)/2) for x in new_r]
    return(ordered_v)


In [4]:
file = 'goetzke_rings.txt'
fws = get_all_fws()
g_frameworks = []
g_tsites = []
for f in fws:
    tsites,osites = get_fw_data(file,f)
    frings = []
    for t in tsites:
        temp = tsites[t]
        temp = temp.split('_')
        temp = [int(n) for n in temp if n !='20']
        g_tsites.append(temp) 
        g_frameworks.append(f)

In [9]:
g_t_list = []
for t in g_tsites:
    temp = [str(x) for x in t]
    temp = '_'.join(temp)
    g_t_list.append(temp)

counts = sorted(Counter(g_t_list).items(),key = lambda item:(-item[1],item[0]))


In [21]:
g_t_list = np.array(g_t_list)
for c in counts[:5]:
    ind = np.where(g_t_list == c[0])[0]
    for i in ind:
        print(g_frameworks[i])
    print(c)
    

IMF
IMF
MEL
MFI
MFI
RRO
SVR
SVR
TUN
TUN
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
('5_5_5_5_5_5_10_10_10_10', 23)
MEL
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
('4_5_5_5_6_6_10_10_10_12_12_12_12', 14)
MEL
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
('4_5_5_5_6_6_8_8_8_8_8_10', 14)
MEL
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
('5_5_5_5_5_6_6_6_10_12', 14)
MEL
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
('5_5_5_5_6_6_6_8_8_10_10_10', 14)


In [36]:
file = 'crum_rings.txt'
fws = get_all_fws()
c_frameworks = []
c_tsites = []
for f in fws:
    tsites,osites = get_fw_data(file,f)
    frings = []
    for t in tsites:
        temp = tsites[t]
        temp = temp.split('_')
        temp = [int(n) for n in temp if n !='20']
        c_tsites.append(temp) 
        c_frameworks.append(f)

In [37]:
c_t_list = []
for t in c_tsites:
    temp = [str(x) for x in t]
    temp = '_'.join(temp)
    c_t_list.append(temp)

counts = sorted(Counter(c_t_list).items(),key = lambda item:(-item[1],item[0]))



In [38]:
c_t_list = np.array(c_t_list)
for c in counts[:5]:
    ind = np.where(c_t_list == c[0])[0]
    for i in ind:
        print(c_frameworks[i])
    print(c)
    

APC
GIS
MER
MWF
MWF
MWF
MWF
MWF
MWF
MWF
MWF
MWF
MWF
MWF
MWF
MWF
MWF
PAU
PAU
PAU
PAU
PAU
PAU
PHI
PHI
PWN
PWN
SIV
SIV
SIV
SIV
('4_4_4_8_8_8_8', 31)
IMF
IMF
IMF
MEL
MFI
MFI
RRO
SVR
SVR
TUN
TUN
TUN
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
('5_5_5_5_5_5_10_10_10_10', 25)
FAR
FRA
FRA
FRA
FRA
FRA
FRA
GIU
LIO
LOS
LOS
LTN
LTN
MAR
SOD
TOL
TOL
('4_4_6_6_6_6', 17)
IMF
MEL
MFI
TUN
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
('5_5_5_5_5_6_6_6_10', 17)
AFS
AFT
AFT
AFT
AFV
AFX
AFX
AVL
AVL
BPH
CHA
GME
SBE
SFW
SFW
SFW
('4_4_4_6_8_8_12', 16)


In [33]:
file = 'sastre_rings.txt'
fws = get_all_fws()
s_frameworks = []
s_tsites = []
for f in fws:
    tsites,osites = get_fw_data(file,f)
    frings = []
    for t in tsites:
        temp = tsites[t]
        temp = temp.split('_')
        temp = [int(n) for n in temp if n !='20']
        s_tsites.append(temp) 
        s_frameworks.append(f)

In [34]:
s_t_list = []
for t in s_tsites:
    temp = [str(x) for x in t]
    temp = '_'.join(temp)
    s_t_list.append(temp)

counts = sorted(Counter(s_t_list).items(),key = lambda item:(-item[1],item[0]))




In [35]:
s_t_list = np.array(s_t_list)
for c in counts[:5]:
    ind = np.where(s_t_list == c[0])[0]
    for i in ind:
        print(s_frameworks[i])
    print(c)

AFG
AFG
AFG
CAN
FAR
FAR
FAR
FAR
FRA
FRA
FRA
FRA
FRA
FRA
GIU
GIU
GIU
GIU
GIU
LIO
LIO
LIO
LIO
LOS
LOS
LTN
LTN
MAR
MAR
MAR
MAR
SOD
TOL
TOL
TOL
TOL
TOL
TOL
TOL
('4_4_6_6_6_6', 39)
IMF
IMF
IMF
MEL
MEL
MFI
MFI
RRO
SVR
SVR
TUN
TUN
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
SFV
('5_5_5_5_5_5_10_10_10_10', 33)
GIS
MER
MWF
MWF
MWF
MWF
MWF
MWF
MWF
MWF
MWF
MWF
MWF
MWF
MWF
MWF
PAU
PAU
PAU
PAU
PAU
PAU
PHI
PHI
PWN
PWN
SIV
SIV
SIV
SIV
('4_4_4_8_8_8_8', 30)
AEI
AEI
AEI
AFT
AFT
AFT
AFV
AFX
AFX
AVL
AVL
CHA
GME
KFI
LTF
MWF
MWF
PAU
PAU
PWN
RHO
SAV
SAV
SAV
SFW
SFW
SFW
TSC
('4_4_4_6_8_8', 28)
AFV
AVE
AVE
AVL
CLO
CLO
EAB
ERI
IFY
IRN
LEV
LTA
LTN
MOZ
OFF
SAT
SWY
SWY
TSC
UFI
PTT
SYT
SYT
SYT
('4_4_4_6_6_8', 24)


In [20]:
from ase.geometry.analysis import Analysis

data = np.loadtxt('iza_vertex_symbols.csv',delimiter=',',dtype = str)
v_tsites = data[:,2]
v_frameworks = data[:,:2]
vertices = dd(list)

counts = sorted(Counter(v_tsites).items(),key = lambda item:(-item[1],item[0]))
for c in counts:
    ind = np.where(v_tsites == c[0])[0]
    for i in ind:
        vertices[c].append(v_frameworks[i])
#         print(v_frameworks[i])
#     print(c)
values = [x for x in vertices.keys()]

ind = 55
print(values[ind])
traj = []
for x in vertices[values[ind]]:
    z = framework(x[0])
    ts,tm,ti = get_tsites(x[0])
    for i,t in enumerate(ts):
        if t==x[1]:
            c,r,ra,a = get_ordered_vertex(z,ti[i],max_ring=10)
            traj += [a]
            ana = Analysis(a)
#             rdf = ana.get_rdf(rmax = 10,nbins = 10,return_dists = True)
#             if x[0] != 'PON':
#                 plt.plot(rdf[0][1],rdf[0][0],label=x)
            print(c,x)
#plt.legend()
#view(traj)
print(len(values))

('4-6-4-6-6-8', 5)
8•6•4•4•6•6 ['ATN' 'T1']
8•6•6•4•4•6 ['JSN' 'T3']
8•6•4•4•6•6 ['PON' 'T1']
8•6•4•4•6•6 ['SAS' 'T1']
8•6•6•4•4•6 ['ZON' 'T3']
649


In [54]:
'''
Vertex Symbol = 4•6•4•6•6•8
'''
checkers = [['ATN','T1'],['SAS','T1'],['JSN','T3'],['ZON','T3']]
for x in checkers:
    ts,tm,ti = get_tsites(x[0])
    for i,t in enumerate(ts):
        if t==x[1]:
            ordered_v = ov(x[0],ti[i])
            print(ordered_v,x)



8•6•4•4•6•6 ['ATN', 'T1']
8•6•4•4•6•6 ['SAS', 'T1']
8•6•6•4•4•6 ['JSN', 'T3']
8•6•6•4•4•6 ['ZON', 'T3']


In [10]:
'''
Vertex Symbol = 4•5_2•5•8_2•5•8_2
'''
checkers = [['MON','T1'],['EON','T9'],['MOR','T3']]
traj = []
for x in checkers:
    ts,tm,ti = get_tsites(x[0])
    z = framework(x[0])
    for i,t in enumerate(ts):
        if t==x[1]:
            ordered_v = ov(x[0],ti[i])
            c,r,ra,a = get_rings(z,ti[i],validation='vertex')
            print(ordered_v,x)
            a,v = center(a,ti[i])
            keep = []
            for path in r:
                for x1 in path:
                    if x1 not in keep:
                        keep.append(x1)
            atoms = a[keep]
            traj+=[atoms]
view(traj)

8_2•8_2•4•5•5_2•5 ['MON', 'T1']
8_2•8_2•5_2•5•4•5 ['EON', 'T9']
8_2•8_2•5_2•5•4•5 ['MOR', 'T3']


In [16]:
sym = 'FER'
z = framework(sym)
ts,tm,ti = get_tsites(sym)
for i,t in enumerate(ts):
    for m in ['goetzke','crum','sastre','vertex']:
        c,r,ra,a = get_rings(z,ti[i],validation=m,max_ring=14)
        print(t,c)

T1 [5, 5, 5, 5, 5, 5, 8, 10, 10, 12, 12, 12, 12, 12]
T1 [5, 5, 5, 5, 5, 5, 8, 10, 10]
T1 [5, 5, 5, 5, 5, 5, 8, 10, 10]
T1 [5, 5, 5, 5, 5, 5, 8]
T2 [5, 5, 5, 5, 6, 8, 8, 10, 10, 10, 10, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12]
T2 [5, 5, 5, 5, 6, 8, 8, 10, 10, 10, 10]
T2 [5, 5, 5, 5, 6, 8, 8, 10, 10, 10, 10, 12, 12]
T2 [5, 5, 5, 5, 6, 8, 8]
T3 [5, 5, 5, 5, 5, 5, 10, 10, 12, 12, 12, 12]
T3 [5, 5, 5, 5, 5, 5, 10, 10]
T3 [5, 5, 5, 5, 5, 5, 10, 10, 12, 12]
T3 [5, 5, 5, 5, 5, 5, 10, 10, 12, 12]
T4 [5, 5, 5, 5, 5, 5, 6, 12, 12, 12, 12, 12, 12, 12, 12]
T4 [5, 5, 5, 5, 5, 5, 6]
T4 [5, 5, 5, 5, 5, 5, 6, 12, 12, 12, 12]
T4 [5, 5, 5, 5, 5, 5, 6]


In [51]:
sym = 'FER'
z = framework(sym)
ts,tm,ti = get_tsites(sym)
traj=[]
for i,t in enumerate(ts):
    if t == 'T2':
        c,r,ra,a = get_rings(z,ti[i],validation='goetzke',max_ring=14)
        print(c)
        new_c = []
        for path in r:
            flag = True
            if len(path) > 12 and len(path)/2-2 in c:
                for p in r:
                    if len(p) == len(path)-4:
#                         print(len(path))
#                         print(path)
#                         print(p)
                        same = [x for x in path if x in p]
                        if len(same) == len(path)/2-1:
                            flag = False
                            bad = path+p
                            break
                        else:
                            flag = True
            if flag:
                new_c.append(int(len(path)/2))
            else:
                atoms,v = center(a,bad[0])
                atoms = atoms[bad]
                traj+=[atoms]
                
        print(new_c)
# view(traj)

[5, 5, 5, 5, 6, 8, 8, 10, 10, 10, 10, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12]
[5, 5, 5, 5, 6, 8, 8, 10, 10, 10, 10, 12, 12, 12, 12, 12, 12, 12, 12]


In [52]:
sym = 'FER'
z = framework(sym)
ts,tm,ti = get_tsites(sym)
for i,t in enumerate(ts):
    if t == 'T2':
        c,r,ra,a = get_rings(z,ti[i],validation='goetzke',max_ring=14)
        print(c)
view(ra)

[5, 5, 5, 5, 6, 8, 8, 10, 10, 10, 10, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12]


In [53]:
sym = 'GME'
z = framework(sym)
ts,tm,ti = get_tsites(sym)
traj=[]
for i,t in enumerate(ts):
    print(t)
    c,r,ra,a = get_rings(z,ti[i],validation='goetzke',max_ring=14)
    print(c)
    new_c = []
    for path in r:
        flag = True
        if len(path) > 12 and len(path)/2-2 in c:
            for p in r:
                if len(p) == len(path)-4:
#                         print(len(path))
#                         print(path)
#                         print(p)
                    same = [x for x in path if x in p]
                    if len(same) == len(path)/2-1:
                        flag = False
                        bad = path+p
                        break
                    else:
                        flag = True
        if flag:
            new_c.append(int(len(path)/2))
        else:
            atoms,v = center(a,bad[0])
            atoms = atoms[bad]
            traj+=[atoms]

    print(new_c)
# view(traj)

T1
[4, 4, 4, 6, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12]
[4, 4, 4, 6, 8, 8, 12, 12, 12, 12, 12, 12, 12]


In [15]:
z = framework('CHA')
ts, tm, ti = get_tsites('CHA')
print(ti)
view(z)
for i in ti:
    c,r,ra,a = get_rings(z,i,validation='vertex')
    print(r)
    keep = []
    for path in r:
        for x in path:
            if x not in keep:
                keep.append(x)
    a,v = center(a,i)
    a = a[keep]
    view(a)

[72]
[[72, 0, 81, 27, 91, 10, 100, 18], [72, 1026, 1293, 267, 960, 63, 81, 0], [72, 1026, 1293, 1200, 1281, 1038, 1056, 1008], [72, 1008, 1056, 978, 1050, 1662, 735, 651, 723, 687, 81, 0], [72, 18, 100, 61, 79, 997, 1076, 1019, 1064, 1000, 1054, 1036, 1069, 1002, 1056, 1008], [72, 1026, 1293, 1211, 1265, 1229, 1271, 1217, 1278, 1041, 87, 33, 94, 49, 100, 18]]


In [16]:
z = framework('CHA')
ts, tm, ti = get_osites('CHA')
print(ti)
view(z)
for i in ti:
    c,r,ra,a = get_rings(z,i,validation='sastre')
    print(c)

[0, 18, 36, 54]
[4, 4, 6]
[4, 8, 8]
[4, 6, 8]
[4, 4, 8]
